In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from string import digits
import re
import os
from sklearn.utils import shuffle
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Input, Dense,Embedding
from keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from keras.preprocessing.text import one_hot
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from nltk.tokenize import RegexpTokenizer

import pickle as pkl
import numpy as np

In [7]:
dir = f'Date/rus-eng.txt'
with open(dir, encoding='utf-8') as f:
    data = f.read()

In [8]:
uncleaned_data_list = data.split('\n')
# uncleaned_data_list = uncleaned_data_list[:38695]

source_word = []
target_word = []
start_target = "sos"
end_target = "eos"

for word in uncleaned_data_list:
    source_word.append(word.split('\t')[0])
    target_word.append(word.split('\t')[1])

In [9]:
language_data = pd.DataFrame(columns=['Source','Target'])
language_data['Source'] = source_word
language_data['Target'] = target_word

In [10]:
# saving to csv
language_data.to_csv(f'{dir}-language_data.csv', index=False)

In [11]:
# loading data from csv
language_data = pd.read_csv(f'{dir}-language_data.csv')

In [12]:
language_data.head()

,Source,Target
0,Марш!,Go.
1,Иди.,Go.
2,Идите.,Go.
3,Здравствуйте.,Hi.
4,Привет!,Hi.


In [13]:
language_data.tail()

,Source,Target
440214,В современном мире перед нами стоит задача дат...,"In today's world, we have to equip all our kid..."
440215,"Смерть - это зачастую то, разговоры или даже м...",Death is something that we're often discourage...
440216,"В тот момент, когда наша экономика растёт, наш...","At a moment when our economy is growing, our b..."
440217,"Поскольку сайтов, посвящённых какой-либо теме,...",Since there are usually multiple websites on a...
440218,"Несомненно, для каждого мужчины в этом мире гд...",Doubtless there exists in this world precisely...


In [14]:
source_word = language_data['Source'].values
target_word = language_data['Target'].values

In [15]:
source_word[0], target_word[0]

('Марш!', 'Go.')

In [16]:
#lowercasing the setences
source_word_ = [x.lower() for x in source_word]
target_word_ = [x.lower() for x in target_word]

In [17]:
source_word_ = [re.sub("'",'',x) for x in source_word_]
target_word_ = [re.sub("'",'',x) for x in target_word_]

In [18]:
source_word_ = [x.strip() for x in source_word_]
target_word_ = [x.strip() for x in target_word_]

In [19]:
source_word_ = [re.sub(r"[^\w\s]", r"", x) for x in source_word_]
target_word_ = [re.sub(r"[^\w\s]", r"", x) for x in target_word_]

In [20]:
source_word_ = [re.sub(r"\d", r"", x) for x in source_word_]
target_word_ = [re.sub(r"\d", r"", x) for x in target_word_]

In [21]:
source_word_ = [re.sub('"', '', x) for x in source_word_]
target_word_ = [re.sub('"', '', x) for x in target_word_]

In [22]:
source_word_ = [re.sub(r"ӕ", r"æ", x) for x in source_word_]

In [23]:
# source_word_ = [f'{start_target} {x} {end_target}' for x in source_word_]
target_word_ = [f'{start_target} {x} {end_target}' for x in target_word_]

In [24]:
source_word_[0], target_word_[0]

('марш', 'sos go eos')

# Data spliting

In [25]:
X = source_word_
Y = target_word_

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1)
len(X_train),len(y_train), len(X_test), len(y_test)

(396197, 396197, 44022, 44022)

In [27]:
X[0], Y[0]

('марш', 'sos go eos')

## Data preparing for encoder and decoder

In [28]:
# preparing data for the word embedding
def Max_length(data):
    max_length_ = max([len(x.split(' ')) for x in data])
    return max_length_

In [29]:
#Training data
max_lenght_source = Max_length(X_train)
max_lenght_target = Max_length(y_train)

#Test data
max_lenght_source_test = Max_length(X_test)
max_lenght_target_test = Max_length(y_test)

In [30]:
max_lenght_target, max_lenght_source

(103, 80)

In [31]:
def tokenizer_(text_data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(text_data)
    return tokenizer

tokenizer_input = tokenizer_(X_train)
vocab_size_input = len(tokenizer_input.word_index) + 1
tokenizer_target = tokenizer_(y_train)
vocab_size_target = len(tokenizer_target.word_index) + 1

In [32]:
with open(f'{dir}-tokenizer_input.pkl','wb') as f:
    pkl.dump(tokenizer_input, f)

with open(f'{dir}-tokenizer_target.pkl','wb') as f:
    pkl.dump(tokenizer_target, f)
    
pkl.dump(tokenizer_input, open(f'{dir}-tokenizer_input.pkl', 'wb'))
pkl.dump(tokenizer_target, open(f'{dir}-tokenizer_target.pkl', 'wb'))

In [33]:
vocab_size_input, vocab_size_target

(54775, 16699)

In [34]:
def generator_batch(X= X_train,Y=y_train, batch_size=128):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_data_input = np.zeros((batch_size,max_lenght_source),dtype='float32') #metrix of batch_size*max_length_english
            decoder_data_input = np.zeros((batch_size,max_lenght_target),dtype='float32') #metrix of batch_size*max_length_marathi
            decoder_target_input = np.zeros((batch_size,max_lenght_target,vocab_size_target),dtype='float32') # 3d array one hot encoder decoder target data
            for i, (input_text,target_text) in enumerate(zip(X[j:j+batch_size],Y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_data_input[i,t] = tokenizer_input.word_index[word] # Here we are storing the encoder 
                                                                         #seq in row here padding is done automaticaly as 
                                                                         #we have defined col as max_lenght
                for t, word in enumerate(target_text.split()):
                    # if word == 'START_':
                    #   word = 'start'
                    # elif word == 'END_':
                    #   word = 'end'
                    decoder_data_input[i,t] = tokenizer_target.word_index[word] # same for the decoder sequence
                    if t>0:
                        decoder_target_input[i,t-1,tokenizer_target.word_index[word]] = 1 #target is one timestep ahead of decoder input because it does not have 'start tag'
            # print(encoder_data_input.shape())
            yield ([encoder_data_input,decoder_data_input],decoder_target_input)

In [35]:
HIDDEN_DIM = 50
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,),name="encoder_inputs")
emb_layer_encoder = Embedding(vocab_size_input,HIDDEN_DIM, mask_zero=True)(encoder_inputs)
encoder = LSTM(HIDDEN_DIM, return_state=True)
encoder_outputs, state_h, state_c = encoder(emb_layer_encoder)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,),name="decoder_inputs")
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
emb_layer_decoder = Embedding(vocab_size_target,HIDDEN_DIM, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(HIDDEN_DIM, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(emb_layer_decoder, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_target, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [37]:
plot_model(model, to_file=f'{dir}-train_model.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [38]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 256
epochs = 120

In [ ]:
# %%capture
model.fit_generator(generator = generator_batch(X_train, y_train, batch_size = batch_size), steps_per_epoch = train_samples//batch_size, epochs=epochs)

C:\Users\ONIGAT~1\AppData\Local\Temp/ipykernel_46384/1782680856.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generator_batch(X_train, y_train, batch_size = batch_size), steps_per_epoch = train_samples//batch_size, epochs=epochs)


Epoch 1/120


In [37]:
model_json = model.to_json()
with open(f'{dir}-model-{epochs}.json', "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(f'{dir}-model-{epochs}_weight.h5')
print("Saved model to disk")

Saved model to disk


In [34]:
json_file = open(f'{dir}-model-{epochs}.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_loaded = model_from_json(loaded_model_json)

model_loaded.load_weights(f'{dir}-model-{epochs}_weight.h5')

In [35]:
encoder_inputs_inf = model_loaded.input[0]
encoder_outputs_inf, inf_state_h, inf_state_c = model_loaded.layers[4].output
encoder_inf_states = [inf_state_h,inf_state_c]
encoder_model = Model(encoder_inputs_inf,encoder_inf_states)

In [36]:
decoder_state_h_input = Input(shape=(HIDDEN_DIM,))
decoder_state_c_input = Input(shape=(HIDDEN_DIM,))
decoder_state_input = [decoder_state_h_input,decoder_state_c_input]

decoder_input_inf = model_loaded.input[1]
decoder_emb_inf = model_loaded.layers[3](decoder_input_inf)
decoder_lstm_inf = model_loaded.layers[5]
decoder_output_inf, decoder_state_h_inf, decoder_state_c_inf = decoder_lstm_inf(decoder_emb_inf, initial_state =decoder_state_input)
decoder_state_inf = [decoder_state_h_inf,decoder_state_c_inf]
dense_inf = model_loaded.layers[6]
decoder_output_final = dense_inf(decoder_output_inf)

decoder_model = Model([decoder_input_inf]+decoder_state_input,[decoder_output_final]+decoder_state_inf)

In [37]:
with open(f'{dir}-tokenizer_input.pkl','rb') as f:
    tokenizer_input = pkl.load(f)
with open(f'{dir}-tokenizer_target.pkl','rb') as f:
    tokenizer_target = pkl.load(f)

reverse_word_map_input = dict(map(reversed, tokenizer_input.word_index.items()))
reverse_word_map_target = dict(map(reversed, tokenizer_target.word_index.items()))

In [38]:
def decode_seq(input_seq):
    state_values_encoder = encoder_model.predict(input_seq)
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tokenizer_target.word_index[start_target]
    stop_condition = False
    decoder_sentance = ''

    while not stop_condition:
        sample_word , decoder_h,decoder_c= decoder_model.predict([target_seq] + state_values_encoder)
        sample_word_index = np.argmax(sample_word[0,-1,:])
        decoder_word = reverse_word_map_target[sample_word_index]
        decoder_sentance += ' '+ decoder_word
        if (decoder_word == end_target or 
            len(decoder_sentance) > 70):
            stop_condition = True
        target_seq[0, 0] = sample_word_index
        state_values_encoder = [decoder_h,decoder_c]
    return decoder_sentance

In [39]:
for i in range(4):
    sentance = X_test[i]
    original_target = y_test[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
    predicted_target = decode_seq(pad_sequence)
    print("Test sentance: ",i+1)
    print("sentance: ",sentance)
    print("origianl translate:",original_target[3:-3])
    print("predicted Translate:",predicted_target[:-3])
    print("=="*50)

Test sentance:  1
sentance:  как ты можешь помочь
origianl translate:  how can you help 
predicted Translate:  you isnt pain you 
Test sentance:  2
sentance:  ты напугал нас
origianl translate:  you scared us 
predicted Translate:  you friends me 
Test sentance:  3
sentance:  я не нервничал
origianl translate:  i wasnt nervous 
predicted Translate:  im ill expert 
Test sentance:  4
sentance:  я сказал сиди здесь
origianl translate:  i said stay here 
predicted Translate:  i hurry smell it 


In [40]:
for i in range(20):
    sentance = X_train[i]
    original_target = y_train[i]
    input_seq = tokenizer_input.texts_to_sequences([sentance])
    pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
    predicted_target = decode_seq(pad_sequence)
    print("Test sentance: ",i+1)
    print("sentance: ",sentance)
    print("origianl translate:",original_target[3:-3])
    print("predicted Translate:",predicted_target[:-3])
    print("=="*50)

Test sentance:  1
sentance:  том встал
origianl translate:  tom got up 
predicted Translate:  tom is mary 
Test sentance:  2
sentance:  куришь
origianl translate:  do you smoke 
predicted Translate:  towel spoon 
Test sentance:  3
sentance:  я свою часть сделал
origianl translate:  i did my part 
predicted Translate:  i leave it 
Test sentance:  4
sentance:  у вас есть дети
origianl translate:  do you have kids 
predicted Translate:  do have it has 
Test sentance:  5
sentance:  тому нас видно
origianl translate:  can tom see us 
predicted Translate:  was tom won 
Test sentance:  6
sentance:  это так глупо
origianl translate:  thats so silly 
predicted Translate:  it was comics 
Test sentance:  7
sentance:  прекрати спорить
origianl translate:  stop arguing 
predicted Translate:  got armed 
Test sentance:  8
sentance:  никто не погиб
origianl translate:  no one is dead 
predicted Translate:  lot is turn 
Test sentance:  9
sentance:  ты ноль
origianl translate:  youre nothing 
predicted 

In [ ]:
# sentance = str(input())
# input_seq = tokenizer_input.texts_to_sequences([sentance])
# pad_sequence = pad_sequences(input_seq, maxlen= 30, padding='post')
# predicted_target = decode_seq(pad_sequence)
# print("predicted Translate:",predicted_target[:-3])